Preparación de Datos

Importar Librerías Necesarias

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

Descargar Recursos de NLTK

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Cargar el Conjunto de Datos

In [ ]:
# Cargarmos el conjunto de datos 20 Newsgroups de la siguiente manera
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
texts = newsgroups.data
labels = newsgroups.target

Preprocesamiento de Texto

In [ ]:
# Definimos lad funcones de preprocesamiento
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    # Eliminar caracteres especiales y números para quedarnos solamente con caracteres permitidos
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenizar
    tokens = text.split()
    # lematizamos y normalizamos
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    text = ' '.join(tokens)
    return text

texts_clean = [preprocess_text(text) for text in texts]

Preprocesamiento de Texto

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(texts_clean, labels, test_size=0.3, random_state=42, stratify=labels)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

Construcción de Embeddings

Entrenar Word2Vec en el Conjunto de Entrenamiento

In [ ]:
from gensim.models import Word2Vec

train_tokens = [text.split() for text in X_train]

# Entrenamos mpdelo Word2Vec, el cual tomará los tokens de las oraciones y
# creará vectores de tamaño 100, con una ventana de contexto de 5 palabras,
# ignorando palabras que aparezcan menos de 2 veces.
w2v_model = Word2Vec(sentences=train_tokens, vector_size=100, window=5, min_count=2, workers=4)

Crear Representaciones de Documentos

In [ ]:
# Función para obtener el vector promedio de un documento
def document_vector(doc):
    # Conservamos solo las palabras que estén en el vocabulario del modelo Word2Vec
    doc = [word for word in doc.split() if word in w2v_model.wv.index_to_key]
    if len(doc) == 0:
        return np.zeros(w2v_model.vector_size)
    else:
        return np.mean(w2v_model.wv[doc], axis=0)

# Obtenemos los vectores de todows los documentos
X_train_vect = np.array([document_vector(doc) for doc in X_train])
X_val_vect = np.array([document_vector(doc) for doc in X_val])
X_test_vect = np.array([document_vector(doc) for doc in X_test])

Implementación del Modelo

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Crear un Pipeline de Preprocesamiento y Modelo

In [ ]:
# Estandarizamos los datos y aplicamos el clasificador
model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, random_state=42))
])


Entrenamiento del Modelo

Entrenar el Modelo con el Conjunto de Entrenamiento

In [ ]:
model.fit(X_train_vect, y_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 SGDClassifier(loss='log_loss', random_state=42))])

Ajuste de Hiperparámetros Usando el Conjunto de Validación

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos el espacio de hiperparámetros a optmizar en el modelko
param_grid = {
    'classifier__alpha': [1e-4, 1e-3, 1e-2],
    'classifier__penalty': ['l2', 'l1', 'elasticnet'],
    'classifier__learning_rate': ['constant', 'optimal', 'adaptive'],
}

# Configuramos GridSearchCV, el cual nos ayudará para explorar diferentes
# combinaciones de hiperparámetros
grid_search = GridSearchCV(model, param_grid, cv=3, n_jobs=-1, scoring='f1_macro')

# Ejecutar búsqueda de hiperparámetros
grid_search.fit(X_train_vect, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
54 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/usr/local/lib/pyth

Evaluación del Modelo

Evaluar el Modelo en el Conjunto de Prueba

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# Predecir en el conjunto de prueba
y_pred = best_model.predict(X_test_vect)

# Mostrar reporte de clasificación
print(classification_report(y_test, y_pred, target_names=newsgroups.target_names))

accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy:.4f}')


                          precision    recall  f1-score   support

             alt.atheism       0.27      0.23      0.25       120
           comp.graphics       0.39      0.43      0.41       146
 comp.os.ms-windows.misc       0.49      0.51      0.50       148
comp.sys.ibm.pc.hardware       0.49      0.35      0.41       147
   comp.sys.mac.hardware       0.45      0.42      0.43       144
          comp.windows.x       0.57      0.57      0.57       148
            misc.forsale       0.59      0.55      0.57       147
               rec.autos       0.33      0.56      0.42       149
         rec.motorcycles       0.33      0.40      0.36       150
      rec.sport.baseball       0.48      0.44      0.46       149
        rec.sport.hockey       0.70      0.64      0.67       150
               sci.crypt       0.62      0.59      0.60       148
         sci.electronics       0.35      0.34      0.35       148
                 sci.med       0.56      0.59      0.58       148
         

Pruebas de Significancia Estadística

 Implementar la Prueba de McNemar

In [ ]:
# Descargar embeddings pre-entrenados de GloVe
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

# Cargamos los embeddings de GloVe en un diccionario
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vectors
    return embeddings_index

glove_embeddings = load_glove_embeddings('glove.6B.100d.txt')


--2024-10-17 19:22:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-10-17 19:22:45--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-17 19:22:45--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

Crear Representaciones de Documentos con GloVe

In [ ]:
# Función para obtener el vector promedio de un documento usando GloVe
def document_vector_glove(doc):
  # Solo nos quedamos con palabras que estén en los embeddings de GloVe
    doc = [word for word in doc.split() if word in glove_embeddings]
    if len(doc) == 0:
        return np.zeros(100)
    else:
        return np.mean([glove_embeddings[word] for word in doc], axis=0)

# Obtener vectores de documentos
X_train_vect_glove = np.array([document_vector_glove(doc) for doc in X_train])
X_val_vect_glove = np.array([document_vector_glove(doc) for doc in X_val])
X_test_vect_glove = np.array([document_vector_glove(doc) for doc in X_test])


Entrenar el Modelo Alternativo

In [ ]:
# Usamos los mismos hiperparámetros encontrados anteriormente y
# el mejor modelo encontrado en la búsqueda anterior, pero ahora con vectores GloVe
model_glove = best_model
model_glove.fit(X_train_vect_glove, y_train)


Evaluar el Modelo Alternativo

In [ ]:
# Predecimos en el conjunto de prueba
y_pred_glove = model_glove.predict(X_test_vect_glove)

print(classification_report(y_test, y_pred_glove, target_names=newsgroups.target_names))

# Precisión general
accuracy_glove = accuracy_score(y_test, y_pred_glove)
print(f'Precisión del modelo con GloVe: {accuracy_glove:.4f}')


Prueba de McNemar

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar

# En esta sección usaremos la prueba de McNemar para comparar ambos modelos

# Crear matriz de contingencia a partir de las predicciones de ambos modelos
contingency_table = confusion_matrix(y_test == y_pred, y_test == y_pred_glove)
print('Matriz de contingencia:')
print(contingency_table)

# Realizamos la prueba de McNemar
result = mcnemar(contingency_table, exact=True)
print('Estadístico de prueba:', result.statistic)
print('P-valor:', result.pvalue)


Análisis e Interpretación de Resultados

Puedes interpretar los resultados de las métricas y la prueba estadística. Un p-valor bajo (por debajo de 0.05) indicaría que hay una diferencia significativa entre los modelos.

 Visualizaciones

Gráficos de Métricas

In [ ]:
import matplotlib.pyplot as plt

# Métricas de los dos modelos
metrics = {
    'Modelo Word2Vec': f1_score(y_test, y_pred, average='macro'),
    'Modelo GloVe': f1_score(y_test, y_pred_glove, average='macro'),
}

# Gráfico de barras para comparar el rendimiento de ambos modelos
plt.bar(metrics.keys(), metrics.values())
plt.ylabel('F1 Score Macro')
plt.title('Comparación de Modelos')
plt.show()


Matriz de Confusión

In [ ]:
import seaborn as sns

# Matriz de confusión para el modelo Word2Vec
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=False, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Modelo Word2Vec')
plt.ylabel('Actual')
plt.xlabel('Predicción')
plt.show()
